<a href="https://colab.research.google.com/github/zekzen/dataclub/blob/main/1_diabet_scikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
İU DATACLUB - Yapay Sinir Aglari Sunumu
07.04.2021
Author: Dr. Zeki Ozen

Dataset kaynagi: UCI Pima-Indian Diabet Dataset @ Kaggle
Link: https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data

Faydalanilan Kod Kaynağı:  https://www.pluralsight.com/guides/machine-learning-neural-networks-scikit-learn

"""

# Gerekli kutuphaneleri calisma ortamimiza dahil edelim
import pandas as pd
import numpy as np

In [2]:
# veri setini calisma ortamimiza yukleyelim

import pandas as pd
url = 'https://raw.githubusercontent.com/zekzen/dataclub/main/diabetes.csv'
df = pd.read_csv(url)

Verisetimizin boyutlarına (satır sütun) bakalım.

In [3]:
print(df.shape)
df.describe().transpose()

(768, 9)


,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [4]:

"""
Input Variables (X):

Number of times pregnant
Plasma glucose concentration a 2 hours in an oral glucose tolerance test
Diastolic blood pressure (mm Hg)
Triceps skin fold thickness (mm)
2-Hour serum insulin (mu U/ml)
Body mass index (weight in kg/(height in m)^2)
Diabetes pedigree function
Age (years)
"""


#bagimli ve bagimsiz degiskenleri ayarlayalim

# y = a1x1 + a2x3 +a3x3  +++ b
# y = aX + b
X = df.iloc[:,:-1]
y = df.iloc[:,-1:]

In [5]:
from sklearn.model_selection import train_test_split
# Veriyi 70-30 oraninda egitim ve test veri seti olarak ikiye ayiralim
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape)
print(X_test.shape)

(537, 8)
(231, 8)


In [6]:
# Veriyi 0-1 araliginda normalize edelim
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [7]:
from sklearn.neural_network import MLPClassifier
# YSA Modelimizi kuralim
mlp = MLPClassifier(hidden_layer_sizes=(10,10,10), 
                    activation='relu', 
                    solver='adam', 
                    max_iter=50000)

In [8]:
# modele egitim veri setini verelim ve egitmeye baslayalim
mlp.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 10, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=50000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [9]:
# modelin egitim ve test veri setlerindeki basarimini degerlendirelim
predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

In [10]:
from sklearn.metrics import classification_report,confusion_matrix
#egitim veri setindeki basarim
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

[[321  37]
 [ 67 112]]
              precision    recall  f1-score   support

           0       0.83      0.90      0.86       358
           1       0.75      0.63      0.68       179

    accuracy                           0.81       537
   macro avg       0.79      0.76      0.77       537
weighted avg       0.80      0.81      0.80       537



In [11]:
# test veri setindeki basarim
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

[[101  41]
 [ 26  63]]
              precision    recall  f1-score   support

           0       0.80      0.71      0.75       142
           1       0.61      0.71      0.65        89

    accuracy                           0.71       231
   macro avg       0.70      0.71      0.70       231
weighted avg       0.72      0.71      0.71       231



In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd', 'adam'],
            'max_iter':[50000],
            'hidden_layer_sizes': [
              (10,10,10), (25,5,5)
             ]
        }
       ]

clf = GridSearchCV(MLPClassifier(), param_grid, cv=3, scoring='accuracy')
clf.fit(X,y)


print("Best parameters set found on development set:")
print(clf.best_params_)

Best parameters set found on development set: {'activation': 'relu', 'hidden_layer_sizes': (10, 10, 10), 'max_iter': 50000, 'solver': 'adam'}